In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import date
from scipy import stats
from statsmodels import regression
import seaborn as sns

C:\Users\Thao\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [2]:
from fredapi import Fred
fred = Fred(api_key='57930a1d3dfc065aee60017aed26075c')
data_sp500 = fred.get_series('SP500')

In [3]:
import simfin as sf
from simfin.names import *
sf.set_api_key('free')
sf.set_data_dir('~/simfin_data/')

In [4]:
years = ['Base Year', 'Year 1', 'Year 2', 'Year 3', 'Year 4', 'Year 5', 'Year 6', 'Year 7', 'Year 8', 'Year 9', 'Year 10', 'Terminal Year']

In [5]:
Revenue_Growth_Rates = pd.Series(index=years,dtype=float)
Revenue = pd.Series(index=years,dtype=float)
Operating_Margin = pd.Series(index=years,dtype=float)
Operating_Income = pd.Series(index=years,dtype=float)
Tax_Rate = pd.Series(index=years,dtype=float)
After_Tax_Operating_Income = pd.Series(index=years,dtype=float)
Reinvestment = pd.Series(index=years,dtype=float)
FCFF = pd.Series(index=years,dtype=float)
Cost_of_Capital = pd.Series(index=years,dtype=float)
Cumulated_Discount_Factor = pd.Series(index=years,dtype=float)
PV_of_FCFF= pd.Series(index=years,dtype=float)

In [6]:
#Collecting Server Data

In [7]:
df_income = sf.load_income(variant='annual', market='us')

Dataset "us-income-annual" on disk (14 days old).
- Loading from disk ... Done!


In [8]:
df_balance=sf.load_balance(variant='annual', market='us')

Dataset "us-balance-annual" on disk (14 days old).
- Loading from disk ... Done!


In [9]:
df_prices = sf.load_shareprices(variant='daily', market='us')

Dataset "us-shareprices-daily" on disk (8 days old).
- Loading from disk ... Done!


In [10]:
df_income_ttm = sf.load_income(variant='ttm', market='us')

Dataset "us-income-ttm" on disk (4 days old).
- Loading from disk ... Done!


In [11]:
df_balance_ttm=sf.load_balance(variant='ttm', market='us')

Dataset "us-balance-ttm" on disk (14 days old).
- Loading from disk ... Done!


In [12]:
sp500 = data_sp500.loc['2015-05-14':'2019-05-06']

In [13]:
df2 = df_income.groupby(TICKER)[[REVENUE, NET_INCOME]].mean()

In [14]:
stock_list = list(df2.index.values)

In [15]:
Beta={}
for i in stock_list:
    try:
        prices = df_prices.loc[i]['2015-05-14':'2019-05-06']['Adj. Close']
        tickers_ret = prices.pct_change()[1:]
        benchmark_ret = sp500.pct_change()[1:]
        df1 = pd.DataFrame(tickers_ret)
        df2 = pd.DataFrame(benchmark_ret,columns=['SPX Daily Return'])
        data = pd.concat([df1,df2], axis=1).dropna()
        (beta, alpha) = stats.linregress(data['SPX Daily Return'],data['Adj. Close'])[0:2]
        Beta.update({i:beta})
    except:
        pass

C:\Users\Thao\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:107: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
C:\Users\Thao\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:117: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
C:\Users\Thao\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\Thao\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\Thao\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\Thao\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:119: RuntimeWarning: invalid value encountere

In [16]:
Beta_list=list(Beta.items())
df_beta=pd.DataFrame(Beta_list)        
df_beta.rename(columns={0:"Stock",1:"Beta"},inplace=True)
df_beta.set_index('Stock',inplace=True)
df_beta.describe()

,Beta
count,1933.000000
mean,1.650688
std,18.231127
min,-15.102778
25%,0.719390
50%,0.979996
75%,1.221193
max,569.755582


In [17]:
#Collecting Specific Stock Data

In [18]:
ticker = ('FB')

In [19]:
income = df_income.loc[ticker,[REVENUE,OPERATING_INCOME,INTEREST_EXP_NET]].fillna(0)

In [20]:
balance = df_balance.loc[ticker, [TOTAL_EQUITY,ST_DEBT,LT_DEBT,CASH_EQUIV_ST_INVEST,SHARES_DILUTED]].fillna(0)

In [21]:
income_ttm = df_income_ttm.loc[ticker,[REVENUE,OPERATING_INCOME,INTEREST_EXP_NET,DEPR_AMOR]].fillna(0)

In [22]:
balance_ttm = df_balance_ttm.loc[ticker, [TOTAL_EQUITY,ST_DEBT,LT_DEBT,CASH_EQUIV_ST_INVEST,SHARES_DILUTED]].fillna(0)

In [104]:
income_revenue_growth= (income['Revenue']/income['Revenue'].shift(1))-1
Five_year_revenue_growth = income_revenue_growth[-5:].mean()
Revenue_Growth_Std = income_revenue_growth.dropna().std()

In [105]:
Operating_Margin_annual = income['Operating Income (Loss)']/income['Revenue']

In [106]:
Average_Operating_Margin = Operating_Margin_annual.mean()
Average_Five_Year_Operating_Margin = Operating_Margin_annual[-5:].mean()
Operating_Margin_Std = Operating_Margin_annual.std()

In [107]:
# List of Stocks

In [108]:
df_list = df_income.groupby(TICKER)[[REVENUE, NET_INCOME]].mean()
stock_list = list(df_list.index.values)

In [109]:
#Cost of Debt

In [110]:
if income_ttm['Interest Expense, Net'].iloc[-1] == 0:
    Interest_Coverage_Ratio = 5
else:
    Interest_Coverage_Ratio = (income_ttm['Operating Income (Loss)'].iloc[-1]+income_ttm['Depreciation & Amortization'].iloc[-1])/income_ttm['Interest Expense, Net'].iloc[-1]
Interest_Coverage_Ratio

5

In [111]:
if Interest_Coverage_Ratio >= 4.25:
    credit_spread = 0.0108
elif 2.5 < Interest_Coverage_Ratio < 4.25:
    credit_spread = 0.0156
elif 2 <= Interest_Coverage_Ratio <= 2.5:
    credit_spread = 0.024
elif 1.25 < Interest_Coverage_Ratio < 2:
    credit_spread = 0.0515
elif 0.65 < Interest_Coverage_Ratio < 1.25:
    credit_spread = 0.0864
else:
    credit_spread = 0.1512

In [112]:
# Input Data

In [169]:
#Inputs
Revenue['Base Year']= (income_ttm['Revenue'].iloc[-1])/1000000
Operating_Income['Base Year'] = income_ttm['Operating Income (Loss)'].iloc[-1]/1000000
Book_Value_of_Equity = balance_ttm['Total Equity'].iloc[-1]/1000000
Book_Value_of_Debt = balance_ttm['Short Term Debt'].iloc[-1]+balance_ttm['Long Term Debt'].iloc[-1]/1000000
Cash_and_Marketable_Securities = balance_ttm['Cash, Cash Equivalents & Short Term Investments'].iloc[-1]/1000000
Number_of_Shares_Outstanding = balance_ttm['Shares (Diluted)'].iloc[-1]/1000000
Effective_Tax_Rate = 0.25
Operating_Lease = 0
Sales_to_Capital_Ratio_year1_5 = 1.7
Sales_to_Capital_Ratio_year6_10 = 1.2
Probability_Failure = 0.1
Distress_Proceed_as_percentage_of_book_fair_value = 0.5
Revenue_Growth_Rates[1:6] = Five_year_revenue_growth/2
Revenue_Growth_Rates['Terminal Year'] = 0.02
Operating_Margin['Terminal Year'] = Average_Five_Year_Operating_Margin
Deprication_Amortization = income_ttm['Depreciation & Amortization'].iloc[-1]/1000000

In [170]:
#Cost of Capital
Risk_Free_Rate = 0.0059
Market_Return = 0.1
Beta = df_beta.loc[ticker]
Cost_of_Equity = Risk_Free_Rate + Beta*(Market_Return-Risk_Free_Rate)

Total_Debt = Book_Value_of_Debt + Operating_Lease
Pretax_Cost_of_Debt = Risk_Free_Rate + credit_spread

Cost_of_Capital['Base Year'] = Cost_of_Equity*((Book_Value_of_Equity)/(Book_Value_of_Equity + Book_Value_of_Debt+Operating_Lease)) + Pretax_Cost_of_Debt*((Book_Value_of_Debt+Operating_Lease)/(Book_Value_of_Equity + Book_Value_of_Debt + Operating_Lease))
Cost_of_Capital['Base Year']

0.11831449269369539

In [171]:
#Simulation Inputs
Revenue_Growth_Mean = Five_year_revenue_growth
Revenue_Growth_Std
Target_Pretax_Operating_Margin_Year10_Mean = Average_Five_Year_Operating_Margin
Target_Pretax_Operating_Margin_Year10_Std = Operating_Margin_Std 
Num_Simulations = 1000

In [172]:
#FCFE Model

In [173]:
data_FCFE={'Revenue_Growth_Rates':Revenue_Growth_Rates,'Revenue': Revenue,'Operating_Margin': Operating_Margin,'Operating_Income': Operating_Income,
'Tax_Rate': Tax_Rate,'After_Tax_Operating_Income':After_Tax_Operating_Income,'Reinvestment':Reinvestment,'FCFF':FCFF,
'Cost_of_Capital':Cost_of_Capital,'Cumulated_Discount_Factor': Cumulated_Discount_Factor,'PV_of_FCFF':PV_of_FCFF}
df_FCFE = pd.DataFrame(data_FCFE,columns=['Revenue_Growth_Rates','Revenue','Operating_Margin','Operating_Income','Tax_Rate','After_Tax_Operating_Income',
                 'Reinvestment','FCFF','Cost_of_Capital','Cumulated_Discount_Factor','PV_of_FCFF'])
df_FCFE

,Revenue_Growth_Rates,Revenue,Operating_Margin,Operating_Income,Tax_Rate,After_Tax_Operating_Income,Reinvestment,FCFF,Cost_of_Capital,Cumulated_Discount_Factor,PV_of_FCFF
Base Year,NaN,5.894900e+04,0.386453,22781.000000,0.25,17085.750000,NaN,NaN,0.118314,NaN,NaN
Year 1,0.240779,8.733633e+04,0.394786,34479.152101,0.25,25859.364076,16698.426676,9160.937400,0.118314,0.894203,8191.736278
Year 2,0.240779,1.293938e+05,0.403119,52161.109959,0.25,39120.832469,24739.677094,14381.155375,0.118314,0.799599,11499.153523
Year 3,0.240779,1.917043e+05,0.411452,78877.192864,0.25,59157.894648,36653.250907,22504.643741,0.118314,0.715003,16090.898036
Year 4,0.240779,2.840209e+05,0.419786,119227.897979,0.25,89420.923484,54303.893981,35117.029503,0.118314,0.639358,22452.358081
Year 5,0.240779,4.207933e+05,0.428119,180149.538077,0.25,135112.153558,80454.334297,54657.819260,0.118314,0.571716,31248.741724
Year 6,0.389246,5.845854e+05,0.428119,250272.006305,0.25,187704.004729,136493.384402,51210.620327,0.114652,0.512910,26266.434823
Year 7,0.296934,7.581689e+05,0.428119,324586.379701,0.25,243439.784776,144652.927809,98786.856967,0.111721,0.461366,45576.853973
Year 8,0.204623,9.133076e+05,0.428119,391004.201475,0.25,293253.151106,129282.290079,163970.861027,0.109377,0.415878,68191.883718
Year 9,0.112311,1.015883e+06,0.428119,434918.459610,0.25,326188.844708,85479.103457,240709.741251,0.107502,0.375510,90388.942961


In [174]:
for year in range (6,11):
    Revenue_Growth_Rates[year] = Revenue_Growth_Rates[year-1] - (Revenue_Growth_Rates[year-1] - Revenue_Growth_Rates['Terminal Year'])/(12-(year+1))
Revenue_Growth_Rates

Base Year             NaN
Year 1           0.240779
Year 2           0.240779
Year 3           0.240779
Year 4           0.240779
Year 5           0.240779
Year 6           0.196623
Year 7           0.152467
Year 8           0.108311
Year 9           0.064156
Year 10          0.020000
Terminal Year    0.020000
dtype: float64

In [175]:
for year in range(1,12):
    Revenue[year] = Revenue[year-1]*(1+ Revenue_Growth_Rates[year])
Revenue

Base Year         58949.000000
Year 1            73142.662675
Year 2            90753.856777
Year 3           112605.451028
Year 4           139718.443398
Year 5           173359.666405
Year 6           207446.155013
Year 7           239074.891643
Year 8           264969.445509
Year 9           281968.755543
Year 10          287608.130654
Terminal Year    293360.293267
dtype: float64

In [176]:
Operating_Margin['Base Year'] = Operating_Income['Base Year']/Revenue['Base Year']
Operating_Margin['Base Year']

0.3864526963985818

In [177]:
for year in range(1,6):
     Operating_Margin[year] = Operating_Margin[year-1] - (Operating_Margin[year - 1]-Operating_Margin['Terminal Year'])/(6-year)
for year in range(6,11):
    Operating_Margin[year]= Operating_Margin['Terminal Year']
Operating_Margin

Base Year        0.386453
Year 1           0.394786
Year 2           0.403119
Year 3           0.411452
Year 4           0.419786
Year 5           0.428119
Year 6           0.428119
Year 7           0.428119
Year 8           0.428119
Year 9           0.428119
Year 10          0.428119
Terminal Year    0.428119
dtype: float64

In [178]:
for year in range(1,12):
    Operating_Income[year] = Revenue[year]*Operating_Margin[year]
Operating_Income

Base Year         22781.000000
Year 1            28875.693835
Year 2            36584.618162
Year 3            46331.781507
Year 4            58651.792818
Year 5            74218.540291
Year 6            88811.608451
Year 7           102352.466672
Year 8           113438.413185
Year 9           120716.138177
Year 10          123130.460940
Terminal Year    125593.070159
dtype: float64

In [179]:
for year in range(0,12):
    Tax_Rate[year]=Effective_Tax_Rate
Tax_Rate

Base Year        0.25
Year 1           0.25
Year 2           0.25
Year 3           0.25
Year 4           0.25
Year 5           0.25
Year 6           0.25
Year 7           0.25
Year 8           0.25
Year 9           0.25
Year 10          0.25
Terminal Year    0.25
dtype: float64

In [180]:
for year in range(0,12):
    After_Tax_Operating_Income[year] = Operating_Income[year]*(1-Tax_Rate[year])
After_Tax_Operating_Income

Base Year        17085.750000
Year 1           21656.770376
Year 2           27438.463622
Year 3           34748.836130
Year 4           43988.844614
Year 5           55663.905218
Year 6           66608.706339
Year 7           76764.350004
Year 8           85078.809889
Year 9           90537.103633
Year 10          92347.845705
Terminal Year    94194.802619
dtype: float64

In [181]:
for year in range(1,6):
    Reinvestment[year] = (Revenue[year]-Revenue[year-1])/Sales_to_Capital_Ratio_year1_5
for year in range(6,12):
    Reinvestment[year] = (Revenue[year]-Revenue[year-1])/Sales_to_Capital_Ratio_year6_10
Reinvestment

Base Year                 NaN
Year 1            8349.213338
Year 2           10359.525942
Year 3           12853.878971
Year 4           15948.819041
Year 5           19788.954710
Year 6           28405.407174
Year 7           26357.280525
Year 8           21578.794888
Year 9           14166.091695
Year 10           4699.479259
Terminal Year     4793.468844
dtype: float64

In [182]:
for year in range(1,12):
    FCFF[year] = After_Tax_Operating_Income[year]- Reinvestment[year]
FCFF

Base Year                 NaN
Year 1           13307.557038
Year 2           17078.937679
Year 3           21894.957159
Year 4           28040.025572
Year 5           35874.950508
Year 6           38203.299165
Year 7           50407.069479
Year 8           63500.015001
Year 9           76371.011937
Year 10          87648.366446
Terminal Year    89401.333775
dtype: float64

In [183]:
Cost_of_Capital['Terminal Year'] = Market_Return
for year in range(1,6):
    Cost_of_Capital[year]=Cost_of_Capital['Base Year']
for year in range(6,11):
    Cost_of_Capital[year]=Cost_of_Capital[year-1]-(Cost_of_Capital[year-1]-Cost_of_Capital['Terminal Year'])/5
Cost_of_Capital

Base Year        0.118314
Year 1           0.118314
Year 2           0.118314
Year 3           0.118314
Year 4           0.118314
Year 5           0.118314
Year 6           0.114652
Year 7           0.111721
Year 8           0.109377
Year 9           0.107502
Year 10          0.106001
Terminal Year    0.100000
dtype: float64

In [184]:
Cumulated_Discount_Factor[1] = 1/(1+Cost_of_Capital[1])
for year in range(2,11):
    Cumulated_Discount_Factor[year] = Cumulated_Discount_Factor[year-1]*(1/(1+Cost_of_Capital[year]))
Cumulated_Discount_Factor

Base Year             NaN
Year 1           0.894203
Year 2           0.799599
Year 3           0.715003
Year 4           0.639358
Year 5           0.571716
Year 6           0.512910
Year 7           0.461366
Year 8           0.415878
Year 9           0.375510
Year 10          0.339521
Terminal Year         NaN
dtype: float64

In [185]:
Cumulated_Discount_Factor[1] = 1/(1+Cost_of_Capital[1])
for year in range(2,11):
    Cumulated_Discount_Factor[year] = Cumulated_Discount_Factor[year-1]*(1/(1+Cost_of_Capital[year]))
Cumulated_Discount_Factor

Base Year             NaN
Year 1           0.894203
Year 2           0.799599
Year 3           0.715003
Year 4           0.639358
Year 5           0.571716
Year 6           0.512910
Year 7           0.461366
Year 8           0.415878
Year 9           0.375510
Year 10          0.339521
Terminal Year         NaN
dtype: float64

In [186]:
for year in range(1,11):
    PV_of_FCFF[year] = FCFF[year]*Cumulated_Discount_Factor[year]
PV_of_FCFF

Base Year                 NaN
Year 1           11899.655352
Year 2           13656.296818
Year 3           15654.970024
Year 4           17927.618128
Year 5           20510.277907
Year 6           19594.850855
Year 7           23256.086036
Year 8           26408.263102
Year 9           28678.087584
Year 10          29758.417551
Terminal Year             NaN
dtype: float64

In [187]:
FCFF['Terminal Year']

89401.33377519772

In [198]:
Terminal_Value = FCFF['Terminal Year']/ Cost_of_Capital['Terminal Year']
Terminal_Value

894013.3377519771

In [201]:
PV_Terminal_Value = Terminal_Value*Cumulated_Discount_Factor['Year 10']
PV_Terminal_Value

303535.8590233331

In [202]:
FCFF_Over_10Year = sum(PV_of_FCFF[1:11])
FCFF_Over_10Year

207344.523356124

In [203]:
Sum_of_PV = FCFF_Over_10Year + PV_Terminal_Value
Sum_of_PV

510880.3823794571

In [204]:
Proceed_if_firm_fails = Sum_of_PV*Distress_Proceed_as_percentage_of_book_fair_value
Proceed_if_firm_fails

255440.19118972856

In [205]:
Value_Operating_Assets = Sum_of_PV*(1- Probability_Failure) + Proceed_if_firm_fails*Probability_Failure
Value_Operating_Assets

485336.3632604843

In [206]:
Value_of_Equity = Value_Operating_Assets + Cash_and_Marketable_Securities - Book_Value_of_Debt
Value_of_Equity

530579.3632604843

In [207]:
Estimated_Value_per_Share = Value_of_Equity/ Number_of_Shares_Outstanding
Estimated_Value_per_Share

182.8323098761145

In [208]:
data_FCFE={'Revenue_Growth_Rates':Revenue_Growth_Rates,'Revenue': Revenue,'Operating_Margin': Operating_Margin,'Operating_Income': Operating_Income,
'Tax_Rate': Tax_Rate,'After_Tax_Operating_Income':After_Tax_Operating_Income,'Reinvestment':Reinvestment,'FCFF':FCFF,
'Cost_of_Capital':Cost_of_Capital,'Cumulated_Discount_Factor': Cumulated_Discount_Factor,'PV_of_FCFF':PV_of_FCFF}
df_FCFE = pd.DataFrame(data_FCFE,columns=['Revenue_Growth_Rates','Revenue','Operating_Margin','Operating_Income','Tax_Rate','After_Tax_Operating_Income',
                 'Reinvestment','FCFF','Cost_of_Capital','Cumulated_Discount_Factor','PV_of_FCFF'])
df_FCFE

,Revenue_Growth_Rates,Revenue,Operating_Margin,Operating_Income,Tax_Rate,After_Tax_Operating_Income,Reinvestment,FCFF,Cost_of_Capital,Cumulated_Discount_Factor,PV_of_FCFF
Base Year,NaN,58949.000000,0.386453,22781.000000,0.25,17085.750000,NaN,NaN,0.118314,NaN,NaN
Year 1,0.240779,73142.662675,0.394786,28875.693835,0.25,21656.770376,8349.213338,13307.557038,0.118314,0.894203,11899.655352
Year 2,0.240779,90753.856777,0.403119,36584.618162,0.25,27438.463622,10359.525942,17078.937679,0.118314,0.799599,13656.296818
Year 3,0.240779,112605.451028,0.411452,46331.781507,0.25,34748.836130,12853.878971,21894.957159,0.118314,0.715003,15654.970024
Year 4,0.240779,139718.443398,0.419786,58651.792818,0.25,43988.844614,15948.819041,28040.025572,0.118314,0.639358,17927.618128
Year 5,0.240779,173359.666405,0.428119,74218.540291,0.25,55663.905218,19788.954710,35874.950508,0.118314,0.571716,20510.277907
Year 6,0.196623,207446.155013,0.428119,88811.608451,0.25,66608.706339,28405.407174,38203.299165,0.114652,0.512910,19594.850855
Year 7,0.152467,239074.891643,0.428119,102352.466672,0.25,76764.350004,26357.280525,50407.069479,0.111721,0.461366,23256.086036
Year 8,0.108311,264969.445509,0.428119,113438.413185,0.25,85078.809889,21578.794888,63500.015001,0.109377,0.415878,26408.263102
Year 9,0.064156,281968.755543,0.428119,120716.138177,0.25,90537.103633,14166.091695,76371.011937,0.107502,0.375510,28678.087584
